# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 4. Ollama REST API - OpenAI  Compatibility</center>

&emsp;&emsp;本节内容我们来看一下 `OpenAI Compatibility`。 `OpenAI` 的 `API` 接口是大模型应用开发中最常用、且集成度最高的 `API` 接口规范，其兼容接口主要包括：

- `chat/completions`
- `completions`
- `models`
- `embeddings`

&emsp;&emsp;我们上两节课程内容中介绍的`/api/generate` 和 `/api/chat` 接口，其实就是 `Ollama` 兼容 `OpenAI` 的 `REST API` 接口的底层实现。其中:

- `/api/generate` 接口对应 `OpenAI` 的 `completions` 接口；
- `/api/chat` 接口对应 `OpenAI` 的 `chat/completions` 接口；

&emsp;&emsp;因此我们现在再来看`ollama` 中的`OpenAI compatibility` 的 `API` 接口调用，就非常容易理解了。


- **非流式调用**

In [ ]:
from openai import OpenAI # type: ignore

client = OpenAI(
    base_url='http://127.0.0.1:11434/v1/',     # 这里修改成可访问的 IP
    api_key='ollama',   # 这里随便写，但是api_key字段一定要有
)

chat_completion = client.chat.completions.create(
    model='deepseek-r1:1.5b',
    messages=[
        {
            'role': 'user',
            'content': '你好，请你介绍一下什么是人工智能？',
        }
    ],
)
print(chat_completion.choices[0].message.content)

ChatCompletion(id='chatcmpl-459', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<think>\n嗯，用户问的是“你是想了解什么是人工智能吗？我给你一个简短的介绍”，然后让我继续。看来他是在考虑以后学习或者工作中是否有涉及到AI的问题，所以对这个概念有些兴趣。\n\n首先我要明白，AI，全称 Artificial Intelligence，是指 machines能够执行人 intelligence tasks 之类的。这可能包括理解语言、处理图像、生成文字等。但我觉得他可能还不太了解，所以我需要简单介绍一下基础的概念以及它的应用。\n\n然后，我应该提到机器学习，因为它是一种算法，可以用来训练模型来模拟和超越人类的智力水平。另外还要强调数据的重要性，因为AI的成功离不开大量的数据和好的算法设计。然后，我可以将机器学习与深度学习联系起来，说明后者是前者的深化和扩展，特别是当有大量复杂信息比如图像或视频时。\n\n最后，总结一下 AI 的应用领域，分为人机交互、数据分析、自动驾驶等，以及它带来的机会和技术挑战，这样用户在决定是不是转向AI或者利用AI时会有更清晰的认识。\n</think>\n\n人工智能（Artificial Intelligence, AI）是一门研究可以用计算机系统模拟人类智能的学科。它可以理解为一种以计算机系统为元组的智能系统。智能系统能够理解和学习，并有自己的判断力和创造力。\n\n### 基本概念\n1. **人机交互**：AI通过自然语言、图像识别、语音合成等技术与人类进行对话、分析和控制。\n2. **数据分析**：涉及机器处理大量的数据，提供洞见（knowledge）以帮助人类理解和改善其环境。\n3. **自动驾驶**：基于实时的位置感知、传感器数据以及复杂模型的系统实现完全自动的汽车。\n\n### 主要研究方向\n- **人机交互领域的应用和学习理论**\n  - 设计更自然人机交互的平台，例如拼音输入框或语音合成工具。\n- **自动化决策**\n  - 设计更可靠的机器智能判断机构，如判断客户忠诚度、预测市场趋势等。\n- **机器学习**

- **流式调用**


In [6]:
from openai import OpenAI

client = OpenAI(
    base_url='http://192.168.110.131:11434/v1/',
    api_key='ollama',
)

messages = [
    {
        'role': 'user',
        'content': '你好，请你介绍一下什么是人工智能？',
    }
]

try:
    # 调用聊天接口
    stream = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        stream=True
    )
    
    # 处理流式响应
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end='', flush=True)
            
except Exception as e:
    print(f"发生错误: {str(e)}")

<think>
嗯，我现在想了解一下什么是人工智能。这个话题好像挺热的，但具体是什么呢？我记得以前看到过一些关于机器学习和深度学习的东西，不过可能还有其他方面。

首先，人工智能听起来像是让机器像人一样思考或者做出决策。以前科幻电影里经常出现机器人取代人类的情节，这应该就是人工智能的一种表现吧。那是不是说所有有智能的机器都是人工智能呢？比如手机里的语音助手，它们能和我对话，可能也是用了AI技术。

我还记得有个词叫做“机器学习”，好像这是实现人工智能的一种方法。机器学习让计算机通过数据自己学习，而不是被明确编程去做某件事。这就像是教小孩认字，不是直接告诉每个字的规则，而是让他们多看多练，自己记住这些字一样。那深度学习又是怎么回事呢？是不是更复杂一点的技术，比如用神经网络来模拟人的大脑？

然后还有自然语言处理和计算机视觉，这些都是人工智能的应用领域吧。自然语言处理就是让机器理解人类的语言，比如翻译软件或者聊天机器人。计算机视觉则是让机器能够识别图像或视频内容，例如人脸识别或者自动驾驶中的目标检测。

人工智能好像有不同的层次，有弱人工智能（ Narrow AI）和强人工智能（General AI）。弱人工智能只能执行特定任务，比如说Siri只会回答问题或设置闹钟，而不能像人类那样全面思考。强人工智能则理论上可以处理各种任务，像电影里的机器人那样，但现实里好像还没实现。

应用方面，除了刚才提到的语音助手和推荐系统，医疗诊断也是个例子。比如AI帮助医生分析X光片或者CT扫描结果，提高诊断准确率。自动驾驶汽车也是一个典型的应用，依靠大量的传感器和算法来做出驾驶决策。

伦理问题可能涉及隐私、就业影响以及AI失控的可能性吧。隐私的话，比如面部识别可能会被用来监视人们；就业方面，自动化可能导致一些岗位的消失；至于AI失控，虽然现在看起来不太现实，但长远来看还是需要考虑的风险。

人工智能的发展离不开大数据和计算能力的进步，这些给了算法更多的数据去学习和处理更复杂的问题。这也让模型变得更大、更高效，推动了技术的发展。

学习人工智能的话，似乎得掌握编程语言如Python或Java，数学知识比如线性代数和微积分也很重要。机器学习的基本概念和一些框架，如TensorFlow或者PyTorch，也是必须要了解的工具。如果我能掌握这些基础，可能就能开始做些AI项目了。

不过，我还不

&emsp;&emsp;在`OpenAI Compatibility` 规范下，目前`Ollama` 支持的模型参数如下：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>支持的功能</font></p>
<div class="center">

| 功能                     | 描述               |
|--------------------------|--------------------|
| 聊天完成                 | Chat completions    |
| 流媒体                   | Streaming           |
| JSON模式                 | JSON mode           |
| 可再现的输出             | Reproducible outputs |
| 视觉                     | Vision              |
| 工具                     | Tools               |

</div>

<p align="center"><font face="黑体" size=4>支持的请求字段</font></p>
<div class="center">

| 请求字段                | 描述                     |
|-------------------------|--------------------------|
| model                   | 模型                     |
| messages                | 消息                     |
| frequency_penalty       | 频率惩罚                 |
| presence_penalty        | 存在惩罚                 |
| response_format         | 响应格式                 |
| seed                    | 种子                     |
| stop                    | 停止                     |
| stream                  | 流式输出                 |
| stream_options          | 流式选项                 |
| include_usage           | 包含使用情况             |
| temperature             | 温度                     |
| top_p                  | Top-p 采样              |
| max_tokens              | 最大令牌数               |
| tools                   | 工具                     |
</div>

In [10]:
from openai import OpenAI

client = OpenAI(
    base_url='http://192.168.110.131:11434/v1/',
    api_key='ollama',
)

messages = [
    {
        'role': 'user',
        'content': '你好，请你介绍一下什么是人工智能？',
    }
]

try:
    # 调用聊天接口
    stream = client.chat.completions.create(
        model='deepseek-r1:32b',
        messages=messages,
        stream=True,
        temperature=0.7,
        max_tokens=1024,
    )
    
    # 处理流式响应
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end='', flush=True)
            
except Exception as e:
    print(f"发生错误: {str(e)}")

<think>
好的，我现在要帮用户介绍什么是人工智能。首先，人工智能这个词大家可能听过不少，但具体是什么意思呢？我应该从基本概念开始讲起。

我记得AI英文是Artificial Intelligence，中文翻译过来就是人工智能。它主要是模拟人类的智能行为，比如学习、推理和解决问题这些。接下来，我可以分几个方面详细说明，这样结构更清晰。

第一部分可以介绍定义和发展背景。人工智能不仅仅是计算机程序，而是能够执行复杂任务的技术集合。还要提到它的起源，比如1956年的达特茅斯会议，还有近年来技术进步带来的突破，比如深度学习和神经网络的应用。

第二部分讲核心能力比较合适。这部分要涵盖感知与理解、思考与推理、学习与适应以及行动与创造这几个方面。每个点都要简单解释一下，让用户明白AI具体能做什么。

第三部分是应用领域，用户可能想知道AI在哪些地方用到了。比如自然语言处理、计算机视觉、机器学习和机器人这些，都是比较热门的应用方向。

第四部分需要说明人工智能的特点。数据驱动、适应性强、自动化程度高、多学科交叉这些都是重要的特点，要逐一解释清楚，让用户明白为什么AI会有这样的表现。

最后，在结尾部分可以总结一下，指出AI已经渗透到各个领域，并且未来可能会带来更多的变化和挑战。

整体上，我要确保内容全面，但不过于技术化，用通俗易懂的语言来描述。这样用户即使没有相关背景也能理解什么是人工智能。
</think>

当然！人工智能（Artificial Intelligence，简称AI）是指由人创造的能够执行通常需要人类智能的任务的系统或机器。这些任务包括学习、推理、问题解决、感知和语言理解等。

### 一、定义与发展背景
人工智能的目标是模拟人类的智能行为，使其能够完成复杂的任务。它起源于20世纪50年代，近年来随着计算能力的提升和大数据的应用，取得了显著进展。

### 二、核心能力
1. **感知与理解**：通过传感器或摄像头捕捉信息，并进行分析。
2. **思考与推理**：基于已有知识进行判断和决策。
3. **学习与适应**：从经验中获取知识并优化性能。
4. **行动与创造**：根据任务要求执行操作或生成内容。

### 三、应用领域
1. **自然语言处理（NLP）**：如聊天机器人、翻译工具。
2. **计算机视觉**：图像识别、面部识别等技术。
3. 